In [5]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [6]:
"""
client = MlflowClient()
versions=[1, 2, 3, 4]
for version in versions:
    client.delete_model_version(name="nyc-taxi-regressor", version=version)
    """

'\nclient = MlflowClient()\nversions=[1, 2, 3, 4]\nfor version in versions:\n    client.delete_model_version(name="nyc-taxi-regressor", version=version)\n    '

In [7]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

2025/07/13 06:24:56 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/13 06:24:56 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


[<Experiment: artifact_location='/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/mlruns/3', creation_time=1752305490359, experiment_id='3', last_update_time=1752305490359, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1751630361732, experiment_id='1', last_update_time=1751630361732, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/mlruns/0', creation_time=1751630361721, experiment_id='0', last_update_time=1751630361721, lifecycle_stage='active', name='Default', tags={}>]

In [8]:
#client.create_experiment(name="my-cool-experiment")

In [9]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [10]:
runs

[<Run: data=<RunData: metrics={'rmse': 6.382890997561556}, params={'learning_rate': '0.762462343335014',
  'max_depth': '71',
  'min_child_weight': '2.114707949297767',
  'objective': 'reg:linear',
  'reg_alpha': '0.027072086783107102',
  'reg_lambda': '0.1387828814896677',
  'seed': '42'}, tags={'mlflow.runName': 'youthful-shark-756',
  'mlflow.source.name': '/home/ubuntu/miniconda3/envs/exp-tracking-env/lib/python3.10/site-packages/ipykernel_launcher.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'ubuntu',
  'model': 'xgboost'}>, info=<RunInfo: artifact_uri='/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/mlruns/1/44fce2fd7ec8412895e0381552db6373/artifacts', end_time=1751674416997, experiment_id='1', lifecycle_stage='active', run_id='44fce2fd7ec8412895e0381552db6373', run_name='youthful-shark-756', start_time=1751674265505, status='FINISHED', user_id='ubuntu'>, inputs=<RunInputs: dataset_inputs=[], model_inputs=[]>, outputs=None>,
 <Run: data=<RunData: metrics={'best_itera

In [11]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 44fce2fd7ec8412895e0381552db6373, rmse: 6.3829
run id: 85d9455d486a4df195dfce41dcc8fefe, rmse: 6.4466
run id: d3d257171995453899d10e4850edd0b0, rmse: 6.4492
run id: 417387225cdf48b1acbe0ed63ffdfc25, rmse: 6.4650
run id: 6f8eb0a18c0c4c0694b4edeff42c92c8, rmse: 6.4802


In [12]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [15]:
run_id = "88e60341de4a4d628f04a1b19aa02885"
model_uri = f"runs:/{run_id}/models_mlflow"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2025/07/13 06:26:52 WARNING mlflow.tracking._model_registry.fluent: Run with id 88e60341de4a4d628f04a1b19aa02885 has no artifacts at artifact path 'models_mlflow', registering model based on models:/m-f7cf7ab287ff469798fa2c5cfca4e58b instead
Created version '10' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1752388012766, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1752388012766, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='88e60341de4a4d628f04a1b19aa02885', run_link=None, source='models:/m-f7cf7ab287ff469798fa2c5cfca4e58b', status='READY', status_message=None, tags={}, user_id=None, version=10>

In [16]:
model_uri

'runs:/88e60341de4a4d628f04a1b19aa02885/models_mlflow'

In [22]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

/tmp/ipykernel_589944/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


version: 9, stage: Staging
version: 10, stage: None


In [23]:
model_version = 10
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_589944/1504810992.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1752388012766, current_stage='Staging', deployment_job_state=None, description=None, last_updated_timestamp=1752389080867, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='88e60341de4a4d628f04a1b19aa02885', run_link=None, source='models:/m-f7cf7ab287ff469798fa2c5cfca4e58b', status='READY', status_message=None, tags={}, user_id=None, version=10>

In [24]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1752388012766, current_stage='Staging', deployment_job_state=None, description='The model version 10 was transitioned to Staging on 2025-07-13', last_updated_timestamp=1752389243209, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='88e60341de4a4d628f04a1b19aa02885', run_link=None, source='models:/m-f7cf7ab287ff469798fa2c5cfca4e58b', status='READY', status_message=None, tags={}, user_id=None, version=10>

In [33]:
model_version = 9
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_589944/3344094314.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1752387834917, current_stage='Production', deployment_job_state=None, description='', last_updated_timestamp=1752390776057, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='', run_link='', source='/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/mlruns/1/models/m-f7cf7ab287ff469798fa2c5cfca4e58b/artifacts', status='READY', status_message=None, tags={}, user_id=None, version=9>

In [34]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"Moved to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1752387834917, current_stage='Production', deployment_job_state=None, description='Moved to Production on 2025-07-13', last_updated_timestamp=1752390777212, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='', run_link='', source='/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/mlruns/1/models/m-f7cf7ab287ff469798fa2c5cfca4e58b/artifacts', status='READY', status_message=None, tags={}, user_id=None, version=9>

In [46]:
from sklearn.metrics import root_mean_squared_error

In [37]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [38]:
df = read_dataframe("data/green_tripdata_2021-03.csv")

/tmp/ipykernel_589944/3034582247.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [39]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,2.0,2021-03-01 00:05:42,2021-03-01 00:14:03,N,1.0,83,129,1.0,1.56,7.50,...,0.5,0.00,0.00,NaN,0.3,8.80,1.0,1.0,0.0,8.350000
1,2.0,2021-03-01 00:21:03,2021-03-01 00:26:17,N,1.0,243,235,1.0,0.96,6.00,...,0.5,0.00,0.00,NaN,0.3,7.30,2.0,1.0,0.0,5.233333
2,2.0,2021-03-01 00:02:06,2021-03-01 00:22:26,N,1.0,75,242,1.0,9.93,28.00,...,0.5,2.00,0.00,NaN,0.3,31.30,1.0,1.0,0.0,20.333333
3,2.0,2021-03-01 00:24:03,2021-03-01 00:31:43,N,1.0,242,208,1.0,2.57,9.50,...,0.5,0.00,0.00,NaN,0.3,10.80,2.0,1.0,0.0,7.666667
4,1.0,2021-03-01 00:11:10,2021-03-01 00:14:46,N,1.0,41,151,1.0,0.80,5.00,...,0.5,1.85,0.00,NaN,0.3,8.15,1.0,1.0,0.0,3.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83822,NaN,2021-03-18 13:00:00,2021-03-18 13:41:00,NaN,NaN,247,77,NaN,18.32,38.20,...,0.0,0.00,6.12,NaN,0.3,50.12,NaN,NaN,NaN,41.000000
83823,NaN,2021-03-18 13:02:00,2021-03-18 13:33:00,NaN,NaN,85,37,NaN,4.23,25.95,...,0.0,0.00,0.00,NaN,0.3,29.00,NaN,NaN,NaN,31.000000
83824,NaN,2021-03-18 13:07:00,2021-03-18 13:34:00,NaN,NaN,209,42,NaN,10.03,41.11,...,0.0,0.00,0.00,NaN,0.3,44.16,NaN,NaN,NaN,27.000000
83825,NaN,2021-03-18 13:59:00,2021-03-18 14:06:00,NaN,NaN,42,74,NaN,0.84,11.95,...,0.0,0.00,0.00,NaN,0.3,15.00,NaN,NaN,NaN,7.000000


In [40]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [41]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [42]:
X_test = preprocess(df, dv)

In [43]:
target = "duration"
y_test = df[target].values

In [47]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 2.99 s, sys: 0 ns, total: 2.99 s
Wall time: 3.07 s


{'rmse': 6.577335225172821}

In [48]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 3.04 s, sys: 0 ns, total: 3.04 s
Wall time: 3.42 s


{'rmse': 6.577335225172821}

In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=10,
    stage="Production",
    archive_existing_versions=True
)